In [1]:
import sqlite3
import pandas as pd

## La oss ta en kikk på databasen

In [2]:
conn = sqlite3.connect("data.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [t[0] for t in cursor.fetchall()]
print('Databasen inneholder følgende tabeller:')
for t in tables:
    print('\t{}'.format(t))

Databasen inneholder følgende tabeller:
	actors
	survey
	hits


In [3]:
print('Tabellene innholder følgende kolonner:')
for t in tables:
    df = pd.read_sql_query('select * from {} limit 1'.format(t), conn)
    print('{}:\n\t{}\n'.format(t, list(df.columns)))

Tabellene innholder følgende kolonner:
actors:
	['age', 'gender', 'education', 'game', 'computer', 'eye', 'nickname', 'position', 'start', 'starttxt', 'end', 'endtxt', 'crowd', 'startexp0', 'startexp1', 'startexp2', 'endexp0', 'endexp1', 'endexp2', 'tothitsexp0', 'tothitsexp1', 'tothitsexp2', 'keydowns0', 'keydowns1', 'keydowns2', 'tothits', 'valid']

survey:
	['actor', 'experiment', 'mental', 'physical', 'temporal', 'effort', 'performance', 'frustration', 'delay', 'time', 'valid']

hits:
	['actor', 'experiment', 'button', 'time', 'valid']



- "actors" innheolder informasjon på hver deltaker i testen, dvs demografi, når hvert eksperiment startet og hvordan personen gjorde det i hvert eksperiment, det finnes dermed en rad for hver deltaker (N rader). Deltakerens unike id er lagret som "rowid" i tabellen, og samsvarer med "actor" i de andre tabellene
- "survey" inneholder undersøkelsene som hver deltaker gjorde for hvert eksperiment, dermed er det N*3 rader i den tabellen
- "hits" er en tabell som har lagret alle treff gjort i underøkelsen, hvilken deltaker som gjorde den, hvilken eksperiment og knapp som ble trykket og når det skjedde. Her er det mange rader.

In [16]:
# Get all valid actors
actors = pd.read_sql_query('select rowid, * from actors WHERE valid IS 1', conn)
actors.head(2)

,rowid,age,gender,education,game,computer,eye,nickname,position,start,...,endexp1,endexp2,tothitsexp0,tothitsexp1,tothitsexp2,keydowns0,keydowns1,keydowns2,tothits,valid
0,2,30,0,6,1,0,0,FirstTestingTroll,7,1.525724e+09,...,1.525724e+09,1.525724e+09,6,9,21,81,99,226,36,1
1,3,24,0,4,2,0,3,Puch,7,1.525726e+09,...,1.525726e+09,1.525727e+09,10,9,20,169,146,218,39,1


In [18]:
# Create new column names and add to actors
columns = pd.read_sql_query('select * from survey', conn).drop(columns=['actor', 'experiment']).columns
new_columns = []
for exp in [0,1,2]:
    for col in columns:
        new_columns.append('{}_{}'.format(col, exp))
for col in new_columns:
    actors[col] = ''
actors.head(2)

,rowid,age,gender,education,game,computer,eye,nickname,position,start,...,valid_1,mental_2,physical_2,temporal_2,effort_2,performance_2,frustration_2,delay_2,time_2,valid_2
0,2,30,0,6,1,0,0,FirstTestingTroll,7,1.525724e+09,...,,,,,,,,,,
1,3,24,0,4,2,0,3,Puch,7,1.525726e+09,...,,,,,,,,,,


In [19]:
# Get survey data and add to newly created columns
for idx, row in actors.iterrows():
    for exp in [0,1,2]:
        survey_row = pd.read_sql_query('select * from survey WHERE actor IS {actor} AND experiment IS {exp}'
                                       .format(actor=row['rowid'], exp=exp), conn)
        if not survey_row.empty:
            for col in columns:
                value = survey_row[col][0]
                actors['{}_{}'.format(col, exp)][idx] = value

c:\users\marti\envs\jupyter\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
actors.head()

,rowid,age,gender,education,game,computer,eye,nickname,position,start,...,valid_1,mental_2,physical_2,temporal_2,effort_2,performance_2,frustration_2,delay_2,time_2,valid_2
0,2,30,0,6,1,0,0,FirstTestingTroll,7,1.525724e+09,...,1,3,2,5,2,8,2,0,1.52572e+09,1
1,3,24,0,4,2,0,3,Puch,7,1.525726e+09,...,1,3,1,9,4,9,0,0,1.52573e+09,1
2,4,25,0,4,2,0,3,the9ko,7,1.525727e+09,...,1,6,2,7,7,8,2,100,1.52573e+09,1
3,5,24,0,4,0,0,3,Gudleik+Knotten,7,1.525725e+09,...,1,1,1,5,3,9,0,1000,1.52573e+09,1
4,6,25,0,4,3,0,2,smik,7,1.525726e+09,...,1,5,3,7,7,7,2,200,1.52573e+09,1


## Trekke ut informasjon fra en deltaker

La oss si at vi ønsker å trekke ut info for en konkret deltaker, la oss si nr 10

In [21]:
pd.read_sql_query('select * from actors where rowid = 6', conn)

,age,gender,education,game,computer,eye,nickname,position,start,starttxt,...,endexp1,endexp2,tothitsexp0,tothitsexp1,tothitsexp2,keydowns0,keydowns1,keydowns2,tothits,valid
0,25,0,4,3,0,2,smik,7,1.525726e+09,2018-05-07 20:41,...,1.525726e+09,1.525726e+09,8,12,18,116,126,177,38,1


In [22]:
pd.read_sql_query('select * from survey where actor = 6', conn)

,actor,experiment,mental,physical,temporal,effort,performance,frustration,delay,time,valid
0,6,2,5,3,7,7,7,2,200,1.525726e+09,1
1,6,1,7,4,6,6,3,4,350,1.525726e+09,1
2,6,0,9,6,6,8,0,8,600,1.525727e+09,1


In [22]:
pd.read_sql_query('select * from hits where actor = 10', conn)

,actor,experiment,button,time,valid
0,10,1,0,1.525734e+09,1
1,10,1,1,1.525734e+09,1
2,10,1,0,1.525734e+09,1
3,10,1,1,1.525734e+09,1
4,10,1,0,1.525734e+09,1
5,10,2,0,1.525735e+09,1
6,10,2,1,1.525735e+09,1
7,10,2,0,1.525735e+09,1
8,10,2,2,1.525735e+09,1
9,10,2,0,1.525735e+09,1
